In [0]:
import numpy as numpy
import scipy as scipy
import pandas as pd
from sklearn import cluster
import matplotlib.pyplot as plt
import io
import time

In [0]:
#converts a numpy array to a list
def set_to_list(numpy_array):
	List = []
	for item in numpy_array:
		List.append(item.tolist())
	return List

In [0]:
#generate datasets
def GenerateData(uploaded):
 	
	df2 = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))
	#print(df2)
	df2 = df2.drop(['y'], axis = 1)
	df2 = numpy.asarray(df2)
	# x1=numpy.random.randn(50,2)
	# x2x=numpy.random.randn(80,1)+12
	# x2y=numpy.random.randn(80,1)
	# x2=numpy.column_stack((x2x,x2y))
	# x3=numpy.random.randn(100,2)+8
	# x4=numpy.random.randn(120,2)+15
	# z=numpy.concatenate((x1,x2,x3,x4))
	return df2

In [0]:
def expandCluster(PointToExpand, PointNeighbors,Cluster,MinPts,eps,Visited,DistanceMatrix,PointClusterNumber,PointClusterNumberIndex  ):
  Neighbors=[]
  #print(len(PointNeighbors))
  #print(len(Visited))
  #k=0
  #print(DistanceMatrix)
  for i in PointNeighbors:
    #print(i)
    #print(PointNeighbors)
    if Visited[i]==0:
      #print(k)
      
      #k = k + 1
      #print(Visited[i])
      #print("--")
      Visited[i]=1
      Neighbors=numpy.where(DistanceMatrix[i]<=eps)[0]
      if len(Neighbors)>=MinPts:

        # Neighbors merge with PointNeighbors
        for j in Neighbors:
            #print(j)
            try:
              PointNeighbors.index(j)
            except ValueError:
              PointNeighbors.append(j)

    if PointClusterNumber[i]==0:
      Cluster.append(i)
      PointClusterNumber[i]=PointClusterNumberIndex
  
  return

In [0]:
def partition(Data):
  part1 = []
  part2 = []
  part3 = []
  for i in range(len(Data)):
    if Data[i][0] <= 400:
      part1.append(Data[i])
    if Data[i][0] >= 300 and Data[i][0] <= 800:
      part2.append(Data[i])
    if Data[i][0] >= 700:
      part3.append(Data[i])
  
  return part1, part2, part3

In [0]:
def DBSCAN(dataset, eps,MinPts, Boundary_info, local_points_info, partition_no,threadName,delay, DistanceMethod = 'euclidean'):
  
  m,n=dataset.shape
  Visited=numpy.zeros(m,'int')
  Type=numpy.zeros(m)
  ClustersList=[]
  Cluster=[]
  PointClusterNumber = numpy.zeros(m)
  PointClusterNumberIndex = 1
  PointNeighbors=[]
  k = 0
  DistanceMatrix = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(dataset, DistanceMethod))

  
  #print(DistanceMatrix.shape)
  for i in range(m):

    if Visited[i]==0:
      Visited[i]=1
      PointNeighbors = numpy.where(DistanceMatrix[i]<eps)[0]
      #print(len(PointNeighbors))
      if len(PointNeighbors)<MinPts:
        Type[i]=-1              # boundary points
      else:
        for k in range(len(Cluster)):
          Cluster.pop()
        Cluster.append(i)
        PointClusterNumber[i]=PointClusterNumberIndex
        #print(len(PointNeighbors))
        PointNeighbors=list(PointNeighbors)
        expandCluster(dataset[i],PointNeighbors,Cluster,MinPts,eps,Visited,DistanceMatrix,PointClusterNumber,PointClusterNumberIndex)
        # print(Cluster)
        # print("/////////")
        # print(PointNeighbors)
        Cluster.append(PointNeighbors[:])
        ClustersList.append(Cluster[:])
        #print(ClustersList)
        PointClusterNumberIndex=PointClusterNumberIndex+1
        #print("-------")

    bound = [[0,0],0,False,0,0]
    bound[0] = dataset[i]
    bound[1] = partition_no
    if Type[i] == 0:
      bound[2] = True
    bound[3] = PointClusterNumber[i]

    if contains(dataset[i], B_1):
      bound[4] = 1
      Boundary_info.append(bound)
    elif contains(dataset[i], B_2):
      bound[4] = 2
      Boundary_info.append(bound)
    else:
      bound[4] = -1
      local_points_info.append(bound)

  count = 0
  while count < 5:
    time.sleep(delay)
    count += 1
    print ("%s: %s" % ( threadName, time.ctime(time.time()) ))
  return PointClusterNumber

In [0]:
def Reduce_Phase(Boundary_info, local_points_info, B1_min_dist, B2_min_dist):
  for i in range(len(Boundary_info)):
    for j in range(i+1, len(Boundary_info)):
      if Boundary_info[i][0][0] == Boundary_info[j][0][0] and Boundary_info[i][0][1] == Boundary_info[j][0][1]:
        Boundary_info[i][2] = Boundary_info[j][2] = Boundary_info[i][2] or Boundary_info[j][2]

        if Boundary_info[i][2] == False:
          ind = -1

          for k in range(len(B1_min_dist)):
            if B1_min_dist[k][2][0] == Boundary_info[i][0][0] and B1_min_dist[k][2][1] == Boundary_info[i][0][1]:
              ind = k
              x = B1_min_dist[k][1][0]
              y = B1_min_dist[k][1][1]

          for k in range(len(B2_min_dist)):
            if B2_min_dist[k][2][0] == Boundary_info[i][0][0] and B2_min_dist[k][2][1] == Boundary_info[i][0][1]:
              ind = k          
              x = B2_min_dist[k][1][0]
              y = B2_min_dist[k][1][1]
          
          for k in range(len(local_points_info)):
            if x == local_points_info[k][0][0] and y == local_points_info[k][0][1]:
              Boundary_info[i][1] = local_points_info[k][1]  #partition number
              Boundary_info[i][3] = local_points_info[k][3]  #Cluster number
              Boundary_info[j][1] = local_points_info[k][1]  #partition number
              Boundary_info[j][3] = local_points_info[k][3]  #Cluster number

  return Boundary_info

In [0]:
def contains(point, B):
    for bt in B:
        if bt[0] == point[0] and bt[1] == point[1]:
          return True
    return False

In [0]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return len(lst3)

In [0]:
def Union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list 

In [0]:
# Boundary Info : boundary point, partition number, isCore, cluster Number, Boundary Number
def Merge_Phase(Boundary_info):
  final_list = []
  visited = numpy.full(len(Boundary_info), False, dtype=bool)

  for i in range(len(Boundary_info)):
    if visited[i] == True:
      continue

    merged_points = []
    code = "P" + str(Boundary_info[i][1]) + "C" + str(Boundary_info[i][3])
    
    if Boundary_info[i][2] == True:
      merged_points.append(code)
  
    for j in range(i+1, len(Boundary_info)):
      code = ""
      if Boundary_info[i][0][0] == Boundary_info[j][0][0] and Boundary_info[i][0][1] == Boundary_info[j][0][1] and Boundary_info[j][2] == True:
        code = "P" + str(Boundary_info[j][1]) + "C" + str(Boundary_info[j][3])
        merged_points.append(code)
        visited[j] = True  
    
    if len(merged_points) != 0:
      #merged_points = [merged_points, Boundary_info[i][0][0], Boundary_info[i][0][1]]
      final_list.append(merged_points)

  merged = numpy.full(len(final_list), True, dtype=bool)

  for i in range(len(final_list)):
    if merged[i] == True:
      for j in range(i+1, len(final_list)):
        if merged[j] == True:
          if intersection(final_list[i], final_list[j]) > 0:
            merged[j] = False
            final_list[i] = Union(final_list[i], final_list[j])

  final = [] 
  for i in range(len(final_list)):
    if merged[i] == True:
      final.append(final_list[i])
  
  return final
    

In [0]:
def Relabel_phase(Boundary_info, local_points_info, dicti):
  df1 = pd.DataFrame(Boundary_info) 
  df2 = pd.DataFrame(local_points_info) 
  frame = [df1, df2]
  df = pd.concat(frame)
  df = df.drop([2, 4], axis=1)
  df = np.asarray(df)
  #print(df)
  for i in range(len(df)):
    code = "P" + str(df[i][1]) + "C" + str(df[i][2])
    #print(code)
    if code in dicti.keys(): 
      df[i][2] = dicti[code]
  #print(df.shape)

  return df

In [0]:
def dictonary(merged_list):
  dicti = dict()
  label = 10
  for i in range(len(merged_list)):
    for key in merged_list[i]:
      dicti[key] = label
    label = label + 1
  #print(dicti)

  return dicti

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
Data=GenerateData(uploaded)

KeyError: ignored

In [0]:
Data

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
#from sklearn.datasets.samples_generator import make_blobsn_components = 3
#plt.scatter(Noisy_Data[:, 0], Noisy_Data[:, 1])
plt.scatter(Data[:, 0], Data[:, 1])
plt.xlabel("alpha")
plt.ylabel("beta");

In [0]:
part1, part2, part3 = partition(Data)
part1 = numpy.asarray(part1)
#print(part1)
part2 = numpy.asarray(part2)
# print("--------")
# print(part2)
part3 = numpy.asarray(part3)
# print("-------")
# print(part3)

In [0]:
noise=scipy.rand(50,2)*20 -3

In [0]:
noise.shape

In [0]:
Noisy_Data=numpy.concatenate((Data,noise))
size=20

In [0]:
Noisy_Data.shape

In [0]:
B_1 = []
B_2 = []
B_3 = []
for data_point in part1:
  if data_point[0] >= 300 and data_point[0] <= 400:
    #print(data_point)
    B_1.append(data_point)
for data_point in part2:
  if data_point[0] >= 700 and data_point[0] <= 800:
    B_2.append(data_point)
#print(B_1)
#print(B_2)
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st 

A_1 = np.asarray(B_2)
#plt.scatter(A_1[:, 0], A_1[:, 1])
plt.scatter(A_1[:, 0], A_1[:, 1],c='r')
plt.xlabel("x")
plt.ylabel("y");

In [0]:
#Nearest neighbours of boundary points
B1_min_dist = []
for b in B_1:
  min_dist = [99999,[0,0],[b[0],b[1]]]  #struct: min distance, nearsest datapoint, boundary point
  for datapoint in Data:
    if not contains(datapoint, B_1):
      euclidean_distance = np.sqrt( (datapoint[0]-b[0])**2 + (datapoint[1]-b[1])**2 )
      if min_dist[0] > euclidean_distance:
        min_dist[0] = euclidean_distance
        min_dist[1][0] = datapoint[0]
        min_dist[1][0] = datapoint[1]
  
  B1_min_dist.append(min_dist)

B2_min_dist = []
for b in B_2:
  min_dist = [99999,[0,0],[b[0],b[1]]]
  for datapoint in Data:
    if not contains(datapoint, B_2):
      euclidean_distance = np.sqrt( (datapoint[0]-b[0])**2 + (datapoint[1]-b[1])**2 )
      if min_dist[0] > euclidean_distance:
        min_dist[0] = euclidean_distance
        min_dist[1][0] = datapoint[0]
        min_dist[1][0] = datapoint[1]
  
  B2_min_dist.append(min_dist)

In [0]:
#print(B1_min_dist)

In [0]:
#result =DBSCAN(Noisy_Data,Epsilon,MinumumPoints)
# Boundary Info : boundary point, partition number, isCore, cluster Number, Boundary Number
Epsilon=50
MinumumPoints=8

Boundary_info = []
local_points_info = []
result1 =DBSCAN(part1,Epsilon,MinumumPoints,Boundary_info,local_points_info,1)
#print(result1)
#axt = Boundary_info
print(len(Boundary_info), Boundary_info)
result2 =DBSCAN(part2,Epsilon,MinumumPoints,Boundary_info,local_points_info,2)
#print(result2)
#print("-----------------")
#print(len(Boundary_info), Boundary_info)
result3 =DBSCAN(part3,Epsilon,MinumumPoints,Boundary_info,local_points_info,3)
#print(result3)
#print("--------------")
#print(len(Boundary_info), Boundary_info)

In [0]:
import threading

Boundary_info = []
local_points_info = []

t1 = threading.Thread( target=DBSCAN, args=(part1,Epsilon,MinumumPoints,Boundary_info,local_points_info,1,"Thread-1",10, ))
t2 = threading.Thread( target=DBSCAN, args=(part2,Epsilon,MinumumPoints,Boundary_info,local_points_info,2,"Thread-2",15, ))
t3 = threading.Thread( target=DBSCAN, args=(part3,Epsilon,MinumumPoints,Boundary_info,local_points_info,3,"Thread-3",20, ))
#t2 = threading.Thread( DBSCAN, (part2,Epsilon,MinumumPoints,Boundary_info,local_points_info,2, ))
# starting thread 1 
t1.start() 
t2.start() 
t3.start() 
t1.join()
t2.join()
t3.join()

In [0]:
Boundary_info = Reduce_Phase(Boundary_info, local_points_info, B1_min_dist, B2_min_dist)
print(Boundary_info)

In [0]:
merged_list = Merge_Phase(Boundary_info)
merged_list

In [0]:
dicti = dictonary(merged_list)

In [0]:
global_info = Relabel_phase(Boundary_info, local_points_info, dicti)

In [0]:
for i in range(len(global_info)):
  if global_info[i][2] < 10:
    print(global_info[i])

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
#from sklearn.datasets.samples_generator import make_blobsn_components = 3
#plt.scatter(Noisy_Data[:, 0], Noisy_Data[:, 1])
points = []
for i in range(len(global_info)):
  points.append(global_info[i][0])

x = np.asarray(points)
LABEL_COLOR_MAP = {10 : 'r',
                   11 : 'k',
                   12 : 'b',
                   1.0 : 'g',
                   0.0 : 'm'
                   }
label_color = []
for i in range(len(global_info)):
   label_color.append(LABEL_COLOR_MAP[global_info[i][2]])

plt.scatter(x[:, 0], x[:, 1], c=label_color)
plt.xlabel("x")
plt.ylabel("y");

In [0]:
queue = []
for i in range(len(global_info)):
  flag = 1
  for j in range(i+1, len(global_info)):
    if global_info[i][0][0] == global_info[j][0][0] and global_info[i][0][1] == global_info[j][0][1]:
      flag = 0
      break
  if flag != 0:
    queue.append(global_info[i])